In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
def gen_data(size=1000000):
    X = np.array(np.random.choice(2, size=(size,)))
    Y = []
    for i in range(size):
        threshold = 0.5
        if X[i-3] == 1:
            threshold += 0.5
        if X[i-8] == 1:
            threshold -= 0.25
        if np.random.rand() > threshold:
            Y.append(0)
        else:
            Y.append(1)
    return X, np.array(Y)
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """Generates a batch iterator for a dataset."""
    data = np.array(data)
    data_size = len(data)
    if len(data) % batch_size == 0:
        num_batches_per_epoch = int(len(data) / batch_size)
    else:
        num_batches_per_epoch = int(len(data) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            indices = np.random.permutation(np.arange(data_size))
        else:
            indices = np.arange(data_size)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[indices[start_index:end_index]]

In [ ]:
# Generate data
num_steps = 20
n_hidden = 4
batch_size = 100
learning_rate = 0.1

In [ ]:
for batch in batches:
    x_batch, y_batch = zip(*batch)
    x_batch, y_batch = np.array(x_batch), np.array(y_batch)

In [ ]:
## Build graph ##
tf.reset_default_graph()

# input
x_input = tf.placeholder(tf.int32, [batch_size, num_steps])
y_input = tf.placeholder(tf.int32, [batch_size, num_steps])
init_state = tf.zeros([batch_size, n_hidden])

x = tf.one_hot(x_input, 2)
x = tf.unpack(x, axis=1)

# GRU cell
gru_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
outputs, final_state = tf.nn.rnn(gru_cell, x, initial_state = init_state)

# To output
W_out = tf.get_variable('W_out', [n_hidden, 2])
b_out = tf.get_variable('b_out', [2], initializer=tf.constant_initializer(0.0))

logits = [tf.matmul(o, W_out) + b_out for o in outputs]
predictions = [tf.nn.softmax(l) for l in logits]

y_as_list = [tf.squeeze(y, squeeze_dims=[1]) for y in tf.split(1, num_steps, y_input)]
loss_weights = [tf.zeros([batch_size]) if i < 8 else tf.ones([batch_size]) for i in range(num_steps)]

# Cost and training
losses = tf.nn.seq2seq.sequence_loss_by_example(logits, y_as_list, loss_weights)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(total_loss)


In [ ]:
(X,Y) = gen_data()
X = np.reshape(X, [-1, num_steps])
Y = np.reshape(Y, [-1, num_steps])

batches = batch_iter(list(zip(X,Y)), batch_size = batch_size, num_epochs = 2)

acc_loss = 0
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i, batch in enumerate(batches):
        x_batch, y_batch = zip(*batch)
        x_batch, y_batch = np.array(x_batch), np.array(y_batch)
        feed_dict = {x_input : x_batch,
                     y_input : y_batch
                     }
        loss, _ = sess.run([total_loss, train_step], feed_dict=feed_dict)
        acc_loss += loss
        if i % 100 == 0 and i > 0:
            print(acc_loss/100)
            acc_loss = 0